In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import joblib

# Load your list of employee data from a CSV file
employee_data = pd.read_csv('user_data.csv')

# Load the trained model and thresholds
# Load the Random Forest classifier model using joblib
clf = joblib.load('trained_model.pkl')

# Define thresholds for different categories
senior_threshold = 0.7
junior_threshold = 0.3

# Initialize label encoder
label_encoder = LabelEncoder()

# Preprocess the employee data similar to the training data
employee_data['Education Level'] = label_encoder.fit_transform(employee_data['Education Level'])
employee_data['Gender'] = label_encoder.fit_transform(employee_data['Gender'])
employee_data['Current Position of the Job'] = label_encoder.fit_transform(employee_data['Current Position of the Job'])
employee_data['Polling Station'] = label_encoder.fit_transform(employee_data['Polling Station'])
employee_data['Experience'] = employee_data['Experience'].str.extract('(\d+)').astype(float)
employee_data['Current Salary'] = employee_data['Current Salary'].str.replace('[\$,]', '', regex=True).astype(float)

# Function to find suitable candidates based on Province, District, and Polling Division
def find_suitable_candidates(input_row):
    province = input_row['Province']
    district = input_row['District']
    polling_division = input_row['Polling Division']
    
    # Filter out candidates from the same province, district, and polling division
    filtered_data = employee_data[
        (employee_data['Province'] != province) |
        (employee_data['District'] != district) |
        (employee_data['Polling Division'] != polling_division)
    ]
    
    # Predict positions for each filtered candidate
    predictions = []
    for index, row in filtered_data.iterrows():
        new_data = pd.DataFrame({
            'Age': [row['Age']],
            'Education Level': [row['Education Level']],
            'Gender': [row['Gender']],
            'Polling Station': [row['Polling Station']],
            'Current Position of the Job': [row['Current Position of the Job']],
            'Experience': [row['Experience']],
            'Current Salary': [row['Current Salary']]
        })

        new_prediction_probs = clf.predict_proba(new_data)

        if new_prediction_probs[0, 2] >= senior_threshold:
            new_prediction = "Senior"
        elif new_prediction_probs[0, 1] >= junior_threshold:
            new_prediction = "Junior"
        else:
            new_prediction = "Clerk"

        predictions.append(new_prediction)

    return predictions

# Predict positions for each input candidate
predictions_list = []
for index, row in employee_data.iterrows():
    predictions = find_suitable_candidates(row)
    predictions_list.append(predictions)

# Add the predicted positions to the employee_data DataFrame
employee_data['Predicted Positions'] = predictions_list

# Save the employee data with predicted positions to a CSV file
employee_data.to_csv('employee_data_with_predictions.csv', index=False)


/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names seen at fit time, yet now missing:
- District_Anuradhapura
- District_Badulla
- District_Colombo
- District_Galle
- District_Jaffna
- ...

  warnings.warn(message, FutureWarning)


ValueError: X has 7 features, but RandomForestClassifier is expecting 35 features as input.